## 문제 2-3: 학생 정보 구조화 시스템

In [5]:
import os
import json
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [ ]:
# 1. 출력 구조를 정의하는 Pydantic 모델
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")

# 2. Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 3. 프롬프트 템플릿 설정
template = """
당신은 학생의 자기소개 텍스트에서 주요 정보를 추출하는 전문가입니다.
다음 자기소개 텍스트를 분석하여, 요청된 형식에 맞춰 정보를 구조화해주세요.

자기소개 텍스트: "{bio_text}"

{format_instructions}

중요 규칙:
1. 'hobbies' 필드는 취미가 여러 개일 경우 리스트 형태로 추출합니다.
2. 각 필드에 해당하는 정보가 텍스트에 명확히 없으면 최대한 유추하거나 비워둘 수 있습니다.
"""

prompt = ChatPromptTemplate.from_template(template)

# 4. 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

# 5. ChatOpenAI 모델 초기화 (Groq API 사용)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# 6. 체인 구성
chain = prompt | model | parser

# 7. 예시 입력 텍스트
bio_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

try:
    # 8. 체인 실행
    output = chain.invoke({"bio_text": bio_text})

    # 9. 결과 출력 - Pydantic 객체를 딕셔너리로 변환 후 JSON 형태로 출력
    print(json.dumps(output.model_dump(), indent=4, ensure_ascii=False))

except Exception as e:
    print(f"정보 추출 중 오류 발생: {e}")
    print("Groq API 키가 올바르게 설정되었는지 확인해 주세요.")

{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": [
        "게임하기",
        "영화보기",
        "코딩"
    ],
    "goal": "훌륭한 개발자가 되는 것"
}
------------------------------
